In [1]:
import numpy as np
import pandas as pd
from openbb_terminal.sdk import openbb
#from statsforecast.models import AutoARIMA
import requests
import re

import altair as a
from scipy.linalg import svd
#import lineapy

import joblib  #for saving models and preprocessing objects
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
%matplotlib inline

# Turn off warnings and messages 
import warnings
warnings.filterwarnings('ignore')


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

In [2]:
btc_df = openbb.crypto.load(symbol="btc",to_symbol="usd",start_date="2019-01-01",source="YahooFinance")
eth_df = openbb.crypto.load(symbol="eth",to_symbol="usd",start_date="2019-01-01",source="YahooFinance")
xrp_df= openbb.crypto.load(symbol='xrp', to_symbol='usd', start_date='2019-01-01', source="YahooFinance")
# Combine btc and eth dataframes and save under /data. Append current date to the filename. 
# Add a new column identifying which coin the row belongs to. 
btc_df['coin'] = 'btc'
eth_df['coin'] = 'eth'
xrp_df['coin']= 'xrp'
btc_eth_df = pd.concat([btc_df,eth_df],axis=0)

btc_eth_df.tail(5)

,Open,High,Low,Close,Adj Close,Volume,coin
date,,,,,,,
2023-10-20,1567.570923,1628.609009,1562.302856,1604.666870,1604.666870,6747486127,eth
2023-10-21,1604.822266,1641.241821,1593.447998,1629.304443,1629.304443,4212179634,eth
2023-10-22,1629.297729,1667.666626,1623.984741,1663.429932,1663.429932,5248406817,eth
2023-10-23,1674.849243,1794.070435,1663.447998,1765.382690,1765.382690,14362295879,eth
2023-10-24,1766.014648,1852.684448,1758.690063,1784.437500,1784.437500,15888690475,eth


In [ ]:
btc= btc_eth_df[btc_eth_df.coin == 'btc']
btc.tail()

In [ ]:
btc.head()
btc = btc.reset_index()

In [ ]:
btc['date'] = btc['date'].astype(str)  # Convert the column to a string
splitted= btc.date.str.split('-', expand=True)
btc['year']= splitted[0].astype('int')
btc['month']= splitted[1].astype('int')
btc['day']= splitted[2].astype('int')
btc.head()

In [ ]:
btc['is_quarter_end']= np.where(btc['month']%3==0,1,0)
btc.tail()

In [ ]:
btc['open-close']= btc['Open']- btc['Close']
btc['low-high']= btc['Low']- btc['High']
btc['target']= np.where(btc['Close'].shift(-1) > btc.Close, 1, 0)
btc.tail()

In [ ]:
#!pip install xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics 
import warnings 
warnings.filterwarnings('ignore')
features= btc[['open-close', 'low-high', 'is_quarter_end']]
target= btc['target']

scaler= StandardScaler()
features= scaler.fit_transform(features)

X_train, X_valid, Y_train, Y_valid= train_test_split(features, target, test_size=0.1, random_state=202)
print(X_train.shape, X_valid.shape)

In [ ]:
models= [LogisticRegression(), SVC(kernel='poly', probability=True), XGBClassifier()]

for i in range(3):
    models[i].fit(X_train, Y_train)
    
    print(f'{models[i]} :')
    print('Training Acc: ', metrics.roc_auc_score(Y_train, models[i].predict_proba(X_train)[:,1]))
    print('Test Acc :', metrics.roc_auc_score(Y_valid, models[i].predict_proba(X_valid)[:,1]))
    print()

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

for i in range(3):
    cm = confusion_matrix(Y_valid, models[i].predict(X_valid))
    plt.figure(figsize=(5,5))
    plt.title(f'{models[i]}')
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# metrics.ConfusionMatrixDisplay(models[0], X_valid, Y_valid)
# plt.show()

In [ ]:
# Define the time period for RSI calculation (e.g., 14 days)
period = 14

# Calculate daily price changes
btc['Price Change'] = btc['Close'].diff()

# Separate gains (positive changes) and losses (negative changes)
btc['Gain'] = btc['Price Change'].apply(lambda x: x if x > 0 else 0)
btc['Loss'] = btc['Price Change'].apply(lambda x: abs(x) if x < 0 else 0)

# Calculate average gains and average losses over the specified period
btc['Avg Gain'] = btc['Gain'].rolling(window=period).mean()
btc['Avg Loss'] = btc['Loss'].rolling(window=period).mean()

# Calculate RS (Relative Strength)
btc['RS'] = btc['Avg Gain'] / btc['Avg Loss']

# Calculate RSI
btc['RSI'] = 100 - (100 / (1 + btc['RS']))

# Drop rows with missing values (due to the rolling averages)
btc = btc.dropna()

# Save the updated dataset with RSI values
btc.to_csv('updated_dataset_with_RSI.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np


# Sort the DataFrame by date (if it's not already sorted)
btc.sort_values(by='date', inplace=True)

# Calculate Simple Moving Average (SMA)
sma_period = 3  # Adjust as needed
btc['SMA'] = btc['Close'].rolling(window=sma_period).mean()

# Calculate Relative Strength Index (RSI)
rsi_period = 14  # Adjust as needed
delta = btc['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=rsi_period).mean()
avg_loss = loss.rolling(window=rsi_period).mean()
rs = avg_gain / avg_loss
btc['RSI'] = 100 - (100 / (1 + rs))

# Calculate Moving Average Convergence Divergence (MACD)
macd_fast_period = 12  # Adjust as needed
macd_slow_period = 26  # Adjust as needed
signal_period = 9  # Adjust as needed
ema_fast = btc['Close'].ewm(span=macd_fast_period).mean()
ema_slow = btc['Close'].ewm(span=macd_slow_period).mean()
btc['MACD'] = ema_fast - ema_slow
btc['Signal'] = btc['MACD'].ewm(span=signal_period).mean()

# Calculate Bollinger Bands
bb_period = 20  # Adjust as needed
btc['Middle_Band'] = btc['Close'].rolling(window=bb_period).mean()
btc['Upper_Band'] = btc['Middle_Band'] + (2 * btc['Close'].rolling(window=bb_period).std())
btc['Lower_Band'] = btc['Middle_Band'] - (2 * btc['Close'].rolling(window=bb_period).std())

# Calculate Stochastic Oscillator
k_period = 14  # Adjust as needed
d_period = 3  # Adjust as needed
low_min = btc['Low'].rolling(window=k_period).min()
high_max = btc['High'].rolling(window=k_period).max()
btc['%K'] = ((btc['Close'] - low_min) / (high_max - low_min)) * 100
btc['%D'] = btc['%K'].rolling(window=d_period).mean()

# Calculate Average True Range (ATR)
atr_period = 14  # Adjust as needed
tr = np.maximum(np.maximum(btc['High'] - btc['Low'], abs(btc['High'] - btc['Close'].shift(1))),
                abs(btc['Low'] - btc['Close'].shift(1)))
btc['ATR'] = tr.rolling(window=atr_period).mean()

# Print or further analyze the DataFrame with manually calculated technical indicators
btc

In [ ]:
btc_supply_data=openbb.crypto.onchain.btc_supply()
btc_supply_data

In [ ]:

# Rename the 'x' column to 'date' and 'y' column to 'btc_supply'
btc_supply_data.rename(columns={'x': 'date', 'y': 'btc_supply'}, inplace=True)

# Convert the 'date' column to a datetime data type
btc_supply_data['date'] = pd.to_datetime(btc_supply_data['date'])

# Extract only the date part from the 'date' column
btc_supply_data['date'] = btc_supply_data['date'].dt.date

# Now your DataFrame should have 'date' as the date without time and 'btc_supply'
btc_supply_data.head()


In [ ]:
# Convert 'date' columns to datetime objects with the same format
btc['date'] = pd.to_datetime(btc['date'])
btc_supply_data['date'] = pd.to_datetime(btc_supply_data['date'])

# Extract only the date part (remove time)
btc['date'] = btc['date'].dt.date
btc_supply_data['date'] = btc_supply_data['date'].dt.date

# Now, try the merge again
merged_data = btc.merge(btc_supply_data, on='date', how='left', suffixes=('_left', '_right'))
merged_data

In [ ]:
x= openbb.crypto.onchain.btc_transac()
x.rename(columns={'x': 'date', 'y': 'confirmed_transactions'}, inplace=True)
x['date'] = pd.to_datetime(x['date'])
x['date'] = x['date'].dt.date

In [ ]:
# Now, try the merge again
merged_data = merged_data.merge(x, on='date', how='left', suffixes=('_left', '_right'))
merged_data

In [ ]:
help(openbb.economy.treasury_chart)

In [ ]:
# trust= openbb.economy.trust(countries=['united_states'])
# trust.rename(columns={'united_states': 'trust_in_us_gov'}, inplace=True)
# trust.reset_index(inplace=True)
# Convert the 'time' column to a datetime object and extract the year
# trust['year'] = pd.to_datetime(trust['TIME']).dt.year
# trust.tail()

In [ ]:
# # Now, try the merge again
# merged_data = merged_data.merge(trust, on='year', how='left', suffixes=('_left', '_right'))
# merged_data

In [ ]:
merged_data.columns

In [ ]:
treasury= openbb.economy.treasury()
treasury.tail()

In [ ]:
treasury= openbb.economy.treasury()
treasury.reset_index(inplace=True)
import pandas as pd

# Assuming 'date_column' is the name of the column you want to convert
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Now, 'date_column' is converted to a datetime dtype
merged_data = merged_data.merge(treasury, on='date', how='left', suffixes=('_left', '_right'))
merged_data

In [ ]:
# Convert date to datetime
treasury['date'] = pd.to_datetime(treasury['date'])
treasury['date'] = treasury['date'].dt.date
treasury.head()

numeric_cols = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '7y', '10y', '20y', '30y']

# Convert all columns except date to numeric from the treasury dataframe
treasury[numeric_cols] = treasury[numeric_cols].apply(pd.to_numeric, errors='coerce')
treasury.dtypes

# Convert all columns except date to numeric from the treasury dataframe

# treasury = treasury.apply(pd.to_numeric, errors='coerce')
# treasury.head()

In [ ]:
# Visualize treasury yields over time. Only use the treasury dataset  
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the treasury yields over time
plt.figure(figsize=(10, 5))
sns.lineplot(x='date', y='10y', data=treasury)
plt.title('10 Year Treasury Yield Over Time')
plt.xlabel('Date')
plt.ylabel('Yield')
plt.show()


In [ ]:
# Plot the 10 year yield versus the 3 and 6 month yields
plt.figure(figsize=(10, 5))
sns.lineplot(x='date', y='3m', data=treasury, label='3m')
sns.lineplot(x='date', y='5y', data=treasury, label='5y')
sns.lineplot(x='date', y='1y', data=treasury, label='1y')
plt.title('Treasury Yield Curve')
plt.xlabel('Date')
plt.ylabel('Yield')
plt.legend()
plt.show()

In [ ]:
# spending= openbb.economy.spending(countries=['united_states'])
# spending.rename(columns={'united_states':'Spending United States'}, inplace=True)
# spending.reset_index(inplace=True)
# spending['year']= pd.to_datetime(spending['TIME']).dt.year
# merged_data = merged_data.merge(spending, on='year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
# spending= openbb.economy.spending(countries=['united_states'], start_date='2019-01-01')

In [ ]:
# Convert the 'date' column to a datetime object and extract the quarter
merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['quarter_year'] = 'Q' + merged_data['date'].dt.quarter.astype(str) + '_' + merged_data['date'].dt.year.astype(str)
merged_data.quarter_year.value_counts()

In [ ]:
merged_data.shape

In [ ]:
# rgdp= openbb.economy.rgdp(countries=['united_states'])
# rgdp.reset_index(inplace=True)
# rgdp.rename(columns={'united_states': 'Gross domestic product (GDP) US'}, inplace=True)
# # Convert the 'date' column to a datetime object and extract the quarter
# rgdp['TIME'] = pd.to_datetime(rgdp['TIME'])
# rgdp['quarter_year'] = 'Q' + rgdp['TIME'].dt.quarter.astype(str) + '_' + rgdp['TIME'].dt.year.astype(str)

# rgdp.tail(2)

In [ ]:
# merged_data = merged_data.merge(rgdp, on='quarter_year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
# revenue= openbb.economy.revenue(countries=['united_states'])
# revenue.reset_index(inplace=True)
# revenue.rename(columns={'united_states': 'Revenue US'}, inplace=True)
# # Convert the 'date' column to a datetime object and extract the quarter
# revenue['TIME'] = pd.to_datetime(revenue['TIME'])
# revenue['year']= pd.to_datetime(revenue['TIME']).dt.year
# revenue.tail()

In [ ]:
# merged_data = merged_data.merge(revenue, on='year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
index= openbb.economy.index(indices=['sp500', 'nyse', 'crypto200', 'cboe_vix3m'])
index.tail()

In [ ]:
index= openbb.economy.index(indices=['sp500', 'nyse', 'crypto200', 'cboe_vix3m'])
index.reset_index(inplace=True)
index.rename(columns={'crypto200': 'CMC Crypto 200 Index by Solacti', 'cboe_vix3m': 'CBOE S&P 500 3-Month Volatility Index', 'Date': 'date'}, inplace=True)
index

In [ ]:
merged_data = merged_data.merge(index, on='date',how='left', suffixes=('_left', '_right'))
merged_data.shape

In [ ]:
# balance=openbb.economy.balance(countries= ['united_states'])
# balance.reset_index(inplace=True)
# balance.rename(columns={'united_states': 'Balance US'}, inplace=True)
# balance['TIME'] = pd.to_datetime(balance['TIME'])
# balance['year']= pd.to_datetime(balance['TIME']).dt.year
# balance.tail(2)

In [ ]:
# merged_data = merged_data.merge(balance, on='year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
# #Inflation measured by consumer price index (CPI)
# ccpi= openbb.economy.ccpi(countries=['united_states'])
# ccpi.reset_index(inplace=True)
# ccpi.rename(columns={'united_states': 'consumer price index(ccpi inflation measure) US'}, inplace=True)
# # Convert the 'date' column to a datetime object and extract the quarter
# ccpi['TIME'] = pd.to_datetime(ccpi['TIME'])
# ccpi['quarter_year'] = 'Q' + ccpi['TIME'].dt.quarter.astype(str) + '_' + ccpi['TIME'].dt.year.astype(str)
# ccpi.tail()

# Many calls to the openbb.economy model return the following error. 
Error getting data from OECD: <urlopen error [SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1129)>

In [ ]:
ccpi= openbb.economy.ccpi(countries=['united_states'])

In [ ]:
# merged_data = merged_data.merge(ccpi, on='quarter_year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
# debt= openbb.economy.debt(countries=['united_states'])
# debt.reset_index(inplace=True)
# debt.rename(columns={'united_states': 'debt-to-GDP ratio US'}, inplace=True)
# debt['TIME'] = pd.to_datetime(debt['TIME'])
# debt['year']= pd.to_datetime(debt['TIME']).dt.year
# debt.tail(5)

In [ ]:
# merged_data = merged_data.merge(debt, on='year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
#Real gross domestic product (GDP)
# fgdp= openbb.economy.fgdp(countries=['united_states'])
# fgdp.reset_index(inplace=True)
# fgdp.rename(columns={'united_states': 'Real gross domestic product (GDP) US'}, inplace=True)
# # Convert the 'date' column to a datetime object and extract the quarter
# fgdp['TIME'] = pd.to_datetime(fgdp['TIME'])
# fgdp['quarter_year'] = 'Q' + fgdp['TIME'].dt.quarter.astype(str) + '_' + fgdp['TIME'].dt.year.astype(str)
# fgdp.tail(5)

In [ ]:
# merged_data = merged_data.merge(fgdp, on='quarter_year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
# gdp= openbb.economy.gdp(countries=['united_states'])
# gdp.reset_index(inplace=True)
# gdp.rename(columns={'united_states': 'GDP per Capita US'}, inplace=True)
# # Convert the 'date' column to a datetime object and extract the quarter
# gdp['TIME'] = pd.to_datetime(gdp['TIME'])
# gdp['year'] = gdp['TIME'].dt.year
# gdp.tail(5)

In [ ]:
# merged_data = merged_data.merge(gdp, on='year',how='left', suffixes=('_left', '_right'))
# merged_data.shape

In [ ]:
bb= openbb.economy

In [ ]:
data_tuple= bb.macro()
# Extract the DataFrame from the tuple
cpi_df = data_tuple[0]

# Reset the index to make 'date' a column
cpi_df.reset_index(inplace=True)

# Rename the columns
cpi_df.columns = ['date', 'CPI']

# Convert 'date' column to datetime type
cpi_df['date'] = pd.to_datetime(cpi_df['date'])

# Print the DataFrame
cpi_df



In [ ]:
# Plot CPI over time 
plt.figure(figsize=(10, 5))
sns.lineplot(x='date', y='CPI', data=cpi_df)
plt.title('Consumer Price Index Over Time')
plt.xlabel('Date')
plt.ylabel('CPI')
plt.show()

In [ ]:
cpi_df['year_month'] = cpi_df['date'].dt.strftime('%Y-%m')
cpi_df.rename(columns={'CPI': 'US CPI'}, inplace=True)
cpi_df.tail()

In [ ]:
cpi_df['year_month'] = cpi_df['date'].dt.strftime('%Y-%m')
cpi_df.rename(columns={'CPI': 'US CPI'}, inplace=True)
merged_data['year_month']= merged_data['date'].dt.strftime('%Y-%m')

In [ ]:
merged_data = merged_data.merge(cpi_df, on='year_month',how='left', suffixes=('_left', '_right'))
merged_data.shape

In [ ]:
merged_data.rename(columns={'date_left': 'Date'}, inplace=True)

In [ ]:
merged_data.columns
#merged_data.drop(columns={'TIME_right', 'TIME_left', 'date_right'}, inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)

merged_data

In [ ]:
merged_data.isna().sum()

In [ ]:
cry_dd= openbb.crypto.dd
cry_dd

In [ ]:
# Count the number of observations by coin 
merged_data.groupby('coin').count()